In [2]:
import numpy as np
import random
from copy import deepcopy

In [18]:
GRID_SIZE = 5

In [3]:
COLOUR_DICT = {
    0: "One", 1: "Blue", 2: "Dark Blue", 3: "Light Blue", 4: "Red", 5: "Orange"
}

In [4]:
# Maybe we can define the grid as an array of tuples, 1st argument indicating whether it is filled, and the 2nd, indicating the type of colour
INITIAL_GRID = [
    [[0, 1], [0, 5], [0, 4], [0, 2], [0, 3]],
    [[0, 3], [0, 1], [0, 5], [0, 4], [0, 2]],
    [[0, 2], [0, 3], [0, 1], [0, 5], [0, 4]],
    [[0, 4], [0, 2], [0, 3], [0, 1], [0, 5]],
    [[0, 5], [0, 4], [0, 2], [0, 3], [0, 1]]
]

# We can also define the tiles that are filled to place gems in the grid, zeros are used here to indicate that nothing is currently present
INITIAL_TILES = [
    [0], [0,0], [0,0,0], [0,0,0,0], [0,0,0,0,0]
]

In [5]:
# Next, let's make a generator to randomly produce assortments of the tops within the game
# That is, we want to produce 7 4-tuples with varying distributions of colour and this MUST be randomized (There are 100 tiles in total and 20 of each colour)
COLOUR_DICT

{0: 'One', 1: 'Blue', 2: 'Dark Blue', 3: 'Light Blue', 4: 'Red', 5: 'Orange'}

In [6]:
def generate_distributions():
    
    while True: 
        curr_list = []
        for i in range(7):
            assort_tuple = (random.randint(1,5), random.randint(1,5), random.randint(1,5), random.randint(1,5))
            curr_list.append(assort_tuple)
        
        if verify_distribution(curr_list):
            return curr_list

def verify_distribution(nums):
    curr_dict = {}
    for tups in nums:
        for x in tups:
            if x in curr_dict:
                curr_dict[x] += 1
            else:
                curr_dict[x] = 1

    for x in curr_dict:
        if curr_dict[x] > 20:
            return False
        
    return True

def convert_nums_to_colours(nums):
    new_list = []
    for x,y,z,w in nums:
        new_list.append((COLOUR_DICT[x], COLOUR_DICT[y], COLOUR_DICT[x], COLOUR_DICT[w]))
    
    return new_list

In [7]:
random_dist = generate_distributions()
random_dist_colors = convert_nums_to_colours(random_dist)

random_dist_colors

[('Light Blue', 'Light Blue', 'Light Blue', 'Red'),
 ('Blue', 'Light Blue', 'Blue', 'Red'),
 ('Blue', 'Light Blue', 'Blue', 'Orange'),
 ('Light Blue', 'Red', 'Light Blue', 'Red'),
 ('Blue', 'Dark Blue', 'Blue', 'Orange'),
 ('Orange', 'Dark Blue', 'Orange', 'Blue'),
 ('Red', 'Light Blue', 'Red', 'Blue')]

In [8]:
# Remember here that we have to pick A COLOUR amongst the tiles which are present on the top, and the rest of the colors are shoved within the middle
# Need to keep a track of those colours in the middle

In [9]:
# 0 represents the ONE tile
MIDDLE_TILES = [0]

In [10]:
# For predictions, we want an algorithm that will tell the user which of the 7 tiles are best, and of those colours within the tiles, which should be selected.
# Should we simulate choosing each of the tiles and test for the possible points which can be obtained and see which selection maxes the points? 
# This would require simulations of the game
# Let's make a player class for this to represent the information that each player will have

In [22]:
class Player:
    def __init__(self):
        self.grid = deepcopy(INITIAL_GRID)
        self.tiles = deepcopy(INITIAL_TILES)
        self.score = 0
        self.punishment = []
    
    # This function tells the user which row is best to fill out when given a certain number of tiles
    def best_tile_row(self, tiles):
        num_tiles = len(tiles)
        tile_type = tiles[0]
        
        # First: Get the rows in which tiles of the type have not been filed out yet
        avail_rows = []
        for i in range(GRID_SIZE):
            row = self.grid[i]
            for j in range(GRID_SIZE):
                cell = row[j]
                if not cell[0] and cell[1] == tile_type:
                    avail_rows.append(i)
                    break
        
        # Next: are there any rows that are (Empty OR same type tile) AND (same number of slots as number of tiles)
        valid_rows = []
        for row_num in avail_rows:
            tile_row = self.tiles[row_num]
            
            num_empty = 0
            for cell in tile_row:
                if not cell:
                    num_empty += 1
            
            if num_empty == num_tiles:
                valid_rows.append(row_num)
        
        # If there are rows that we can fill then we go ahead and fill them and calculate the score
        if valid_rows:
            
        
        # If there are NONE valid rows then we simply just fill out the first valid row we can (primitive but let's see how it goes)
        else:
            
            
        
        print(valid_rows)
        print(tiles)
            
            
    # The row and column is to tell where the most recent tile has been added so the score can be calculated accordingly
    def get_score(self, grid, row, column):
        
        # Let us check column wise and row wise, how the score will be calculated
        # We go from the tile itself, to the left and check how many consecutively and then to the right consecutively and minus one, same goes for column
        
        score_row = grid[row]
        left_score = 0
        right_score = 0
        for i in range(column-1, -1, -1):
            if not score_row[i][0]:
                break
            left_score += 1
                
        for i in range(column+1, 5):
            if not score_row[i][0]:
                break
            right_score += 1
        
        row_score = left_score + right_score
        
        up_score = 0
        down_score = 0
        score_column = [r[column] for r in grid]
        
        for i in range(row-1, -1, -1):
            if not score_column[i][0]:
                break
            up_score += 1
                
        for i in range(row+1, 5):
            if not score_column[i][0]:
                break
            down_score += 1
        
        column_score = up_score + down_score
        
        total_score = row_score + column_score
        if total_score == 0:
            total_score = 1
            
        return total_score
    
    # This function does not make any changes to the tops nor the number of tiles themselves, it only returns the chosen number of tiles on the selected top
    # The numbers will only be changed after the best possibly outcome is calculated and selected
    def take_colors(self, middles, tiles_arr, tile_colour, TAKE_MIDDLES = False):
        if not TAKE_MIDDLES:
            chosen_tiles = [tile for tile in tiles_arr if tile == tile_colour]
            return chosen_tiles
        
        else:
            chosen_tiles = [tile for tile in middles if tile == tile_colour]
            if 0 in middles:
                middles.remove(0)
                chosen_tiles.append(0)
                self.punishment.append(0)
                
            return chosen_tiles
            
        
        

In [23]:
# Generate the random starting distribution of tiles and begi

test_player1 = Player()
test_player2 = Player()
test_player3 = Player()
player_arr = [test_player1, test_player2, test_player3]
starting_tiles = generate_distributions()
middle_tiles = [0]

In [24]:
test_player1.best_tile_row([3,3,3])
print(test_player1.grid)
print(test_player1.tiles)

[2]
[3, 3, 3]
[[[0, 1], [0, 5], [0, 4], [0, 2], [0, 3]], [[0, 3], [0, 1], [0, 5], [0, 4], [0, 2]], [[0, 2], [0, 3], [0, 1], [0, 5], [0, 4]], [[0, 4], [0, 2], [0, 3], [0, 1], [0, 5]], [[0, 5], [0, 4], [0, 2], [0, 3], [0, 1]]]
[[0], [0, 0], [0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0, 0]]


In [12]:
# With the appropriate functions we can try to simulate what a round of azul might look like, we can take an approach that maximizes our points and minimizing the enemy's points
# Every time a circle of turns pass, we can update the number of tiles left available and repeat the calculations, since it is not set in stone for what the others might do

In [13]:
# Lets perform a test run calculation

def generate_options(tiles, middle_tiles, player):
    options_arr = []
    unique_middle_colors = list(set(middle_tiles))
    if len(unique_middle_colors) > 1:
        for color in unique_middle_colors:
            chosen = player.take_colors(middle_tiles, [], color, TAKE_MIDDLES=True)
            
            # In choosing the middle tiles
            points_earned, best_row = player.best_tile_row(chosen)            
            score_tuple = (points_earned, best_row, color, chosen)

            options_arr.append(score_tuple)
    
    for top in tiles:
        unique_colors = list(set(top))

        for color in unique_colors:
            chosen = player.take_colors(middle_tiles, top, color, TAKE_MIDDLES=False)
        
            # For these chosen tiles, now fill in the tiles and get the expected score
            points_earned, best_row = player.best_tile_row(chosen)
            score_tuple = (points_earned, best_row, color, top)

            options_arr.append(score_tuple)
        
    
    return options_arr
    
options_arr = generate_options(starting_tiles, middle_tiles, test_player1)
options_arr

[(1, [0], 3, (5, 3, 4, 4)),
 (1, [0, 0], 4, (5, 3, 4, 4)),
 (1, [0], 5, (5, 3, 4, 4)),
 (1, [0], 1, (3, 2, 5, 1)),
 (1, [0], 2, (3, 2, 5, 1)),
 (1, [0], 3, (3, 2, 5, 1)),
 (1, [0], 5, (3, 2, 5, 1)),
 (1, [0, 0], 1, (5, 1, 1, 3)),
 (1, [0], 3, (5, 1, 1, 3)),
 (1, [0], 5, (5, 1, 1, 3)),
 (1, [0], 1, (5, 4, 1, 5)),
 (1, [0], 4, (5, 4, 1, 5)),
 (1, [0, 0], 5, (5, 4, 1, 5)),
 (1, [0], 3, (5, 4, 5, 3)),
 (1, [0], 4, (5, 4, 5, 3)),
 (1, [0, 0], 5, (5, 4, 5, 3)),
 (1, [0, 0, 0], 1, (1, 1, 2, 1)),
 (1, [0], 2, (1, 1, 2, 1)),
 (1, [0, 0, 0], 1, (1, 1, 1, 5)),
 (1, [0], 5, (1, 1, 1, 5))]

In [15]:
# Now we can compute a tree of possibilities
# Go through the cycle of players until each possibility has been explored

curr_player_index = 0 # Increase this and mod len of players when we cycle
middle = [0]

for option in options_arr:
    tops_copy = deepcopy(starting_tiles)
    print(option)
    
    # Let's look at the possibility for a current option
    score, row, color, top = option
    
    # From the color selected and the top chosen, we remove it from the list of tops, push the remaining into the middle
    leftover = [tile for tile in top if tile != color]
    middle += leftover
    
    tops_copy.remove(top)
    
    # Now we have a new set of tops and can generate a new set of possibilities
    # Move on to next player
    curr_player += 1
    
    


(1, [0], 3, (5, 3, 4, 4))
(1, [0, 0], 4, (5, 3, 4, 4))
(1, [0], 5, (5, 3, 4, 4))
(1, [0], 1, (3, 2, 5, 1))
(1, [0], 2, (3, 2, 5, 1))
(1, [0], 3, (3, 2, 5, 1))
(1, [0], 5, (3, 2, 5, 1))
(1, [0, 0], 1, (5, 1, 1, 3))
(1, [0], 3, (5, 1, 1, 3))
(1, [0], 5, (5, 1, 1, 3))
(1, [0], 1, (5, 4, 1, 5))
(1, [0], 4, (5, 4, 1, 5))
(1, [0, 0], 5, (5, 4, 1, 5))
(1, [0], 3, (5, 4, 5, 3))
(1, [0], 4, (5, 4, 5, 3))
(1, [0, 0], 5, (5, 4, 5, 3))
(1, [0, 0, 0], 1, (1, 1, 2, 1))
(1, [0], 2, (1, 1, 2, 1))
(1, [0, 0, 0], 1, (1, 1, 1, 5))
(1, [0], 5, (1, 1, 1, 5))
